## Read data: products dataset

In [1]:
import pandas as pd

df_products=pd.read_csv('../data/brazilian_e-commerce/olist_products_dataset.csv')
df_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


## Data inspection

In [2]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [3]:
df_products.isnull().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [4]:
df_products.duplicated().sum()

np.int64(0)

In [5]:
num_unique = df_products['product_id'].nunique()
print("Number of unique product_id:", num_unique)

Number of unique product_id: 32951


In [6]:
df_products['product_category_name'].nunique()

73

## Start Ollama

In [7]:
import requests
import json

# Ollama API base URL
OLLAMA_URL = "http://localhost:11434"
MODEL = "llama3:8b"  # Change this to your installed model

In [8]:
def list_models() -> list:
    """List all available models installed locally."""
    response = requests.get(f"{OLLAMA_URL}/api/tags")
    return [model["name"] for model in response.json()["models"]]

In [9]:
    print("=" * 50)
    print("OLLAMA INTERACTION EXAMPLES")
    print("=" * 50)
    
    # Check if Ollama is running
    try:
        models = list_models()
        print(f"\n✅ Ollama is running!")
        print(f"📦 Available models: {models}\n")
    except requests.exceptions.ConnectionError:
        print("\n❌ Cannot connect to Ollama. Start it with: ollama serve")
        exit(1)

OLLAMA INTERACTION EXAMPLES

✅ Ollama is running!
📦 Available models: ['llama3:latest', 'llama3:8b']



In [10]:
def simple_chat(prompt: str) -> str:
    """
    Send a simple prompt and get a response.
    This is the most basic way to interact with Ollama.
    """
    response = requests.post(
        f"{OLLAMA_URL}/api/generate",
        json={
            "model": MODEL,
            "prompt": prompt,
            "stream": False  # Get the full response at once
        }
    )
    return response.json()["response"]

In [11]:
df_products.loc[ 0, 'product_category_name']

'perfumaria'

In [12]:
# Example 1: Simple one-shot prompt
print("\n--- Example 1: Simple Chat ---")
response = simple_chat("translate "+df_products.loc[ 0, 'product_category_name']+" into english")
print(f"Response: {response}\n")


--- Example 1: Simple Chat ---
Response: The lovely word "Perfumaria"!

After some research, I found that "Perfumaria" is not a commonly used term in English. However, based on its Latin roots and context, here are a few possible translations:

1. Perfume shop: This is the most literal translation, as perfumaria is derived from the Latin words "per fumus," meaning "through smoke" or "fragrance."
2. Fragrance store: Another possible translation, this term could refer to a store that sells fragrances, including perfumes, essential oils, and other scented products.
3. Scented goods: This phrase captures the essence of perfumearia as a place where one can find scented items, such as candles, soaps, and potpourri.

In some contexts, Perfumaria might also be used as a brand name or a fictional location in a story. In this case, the translation would depend on the specific context and purpose.

Please note that these translations are my best guesses based on linguistic analysis. If you have 

## Translate portuguese product category names into english

In [14]:
import pandas as pd
import json
import requests
import math
import time

OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL = "llama3:8b"
BATCH_SIZE = 40   # safe chunk size

df_products = pd.read_csv('../data/brazilian_e-commerce/olist_products_dataset.csv')
categories_pt = df_products['product_category_name'].dropna().unique().tolist()

def ollama_generate(prompt):
    response = requests.post(
        OLLAMA_URL,
        json={"model": MODEL, "prompt": prompt, "temperature": 0},
        stream=True,
    )
    text = ""
    for line in response.iter_lines():
        if line:
            obj = json.loads(line.decode())
            text += obj.get("response", "")
    return text

def translate_batch(batch, start_index):
    indexed_list = [
        {"index": start_index+i, "pt": batch[i]} 
        for i in range(len(batch))
    ]

    prompt = f"""
You MUST translate the Portuguese product category names into English.

Return ONLY a JSON array of objects in the following exact format:

[
  {{"index": 0, "pt": "xxxx", "en": "English here"}},
  ...
]

Rules:
- Keep EXACT same indices.
- ONE object per item.
- NO missing items.
- NO grouping items.
- NO commentary.
- NO extra text.
- NO partial results.

Translate this list:
{json.dumps(indexed_list, ensure_ascii=False)}
"""

    # retry until perfect
    for attempt in range(5):
        raw = ollama_generate(prompt)

        try:
            # Extract between [ ... ]
            start = raw.find("[")
            end = raw.rfind("]")
            json_text = raw[start:end+1]

            data = json.loads(json_text)

            # validation
            if len(data) != len(batch):
                raise ValueError("Length mismatch")

            data_sorted = sorted(data, key=lambda x: x["index"])
            return [d["en"] for d in data_sorted]

        except Exception:
            if attempt == 4:
                raise
            time.sleep(1)

all_translations = []
num_batches = math.ceil(len(categories_pt) / BATCH_SIZE)
for b in range(num_batches):
    batch = categories_pt[b*BATCH_SIZE:(b+1)*BATCH_SIZE]
    print(f"Batch {b+1}/{num_batches} → {len(batch)} items")
    trans = translate_batch(batch, b*BATCH_SIZE)
    all_translations.extend(trans)

translation_map = dict(zip(categories_pt, all_translations))
df_products["product_category_name_english"] = df_products["product_category_name"].map(translation_map)

print(df_products.head())
print("Total categories:", len(categories_pt))
print("Total translations:", len(all_translations))


Batch 1/2 → 40 items
Batch 2/2 → 33 items
                         product_id  product_category_name  \
0  1e9e8ef04dbcff4541ed26657ea517e5             perfumaria   
1  3aa071139cb16b67ca9e5dea641aaa2f                  artes   
2  96bd76ec8810374ed1b65e291975717f          esporte_lazer   
3  cef67bcfe19066a932b7673e239eb23d                  bebes   
4  9dc1a7de274444849c219cff195d0b71  utilidades_domesticas   

   product_name_lenght  product_description_lenght  product_photos_qty  \
0                 40.0                       287.0                 1.0   
1                 44.0                       276.0                 1.0   
2                 46.0                       250.0                 1.0   
3                 27.0                       261.0                 1.0   
4                 37.0                       402.0                 4.0   

   product_weight_g  product_length_cm  product_height_cm  product_width_cm  \
0             225.0               16.0               10.0    

In [21]:
df_products.to_csv(
    '../data/brazilian_e-commerce/prep_products_dataset_translated.csv',
    index=False,
    encoding='utf-8'
)

In [22]:
print(df_products[['product_category_name', 'product_category_name_english']])


                   product_category_name product_category_name_english
0                             perfumaria                     Perfumery
1                                  artes                          Arts
2                          esporte_lazer            Sports and Leisure
3                                  bebes                 Baby Products
4                  utilidades_domesticas          Household Essentials
...                                  ...                           ...
32946                   moveis_decoracao      Furniture and Decoration
32947  construcao_ferramentas_iluminacao   Building Tools and Lighting
32948                    cama_mesa_banho              Bed and Bathroom
32949             informatica_acessorios          Computer Accessories
32950                    cama_mesa_banho              Bed and Bathroom

[32951 rows x 2 columns]


## Data inspection

In [25]:
df_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,Perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,Arts
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,Sports and Leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,Baby Products
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,Household Essentials


In [26]:
df_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,Perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,Arts
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,Sports and Leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,Baby Products
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,Household Essentials


In [23]:
df_products['product_category_name_english'].nunique()

71

In [24]:
df_products['product_category_name_english'].unique()

array(['Perfumery', 'Arts', 'Sports and Leisure', 'Baby Products',
       'Household Essentials', 'Musical Instruments', 'Cool Stuff',
       'Furniture and Decoration', 'Electrical Appliances', 'Toys',
       'Bed and Bathroom', 'Construction Tools and Safety',
       'Computer Accessories', 'Beauty and Health',
       'Travel Bags and Accessories', 'Garden Tools', 'Office Furniture',
       'Automotive', 'Electronics', 'Fashion and Footwear', 'Telephony',
       'Stationery', 'Fashion Accessories and Bags', 'PCs',
       'Home Construction', 'Watches and Gifts',
       'Construction Tools for Construction', 'Pet Shop',
       'Agriculture, Industry, and Commerce', nan,
       'Living Room Furniture', 'Signage and Security',
       'Air Conditioning', 'Consoles and Games',
       'Books and General Interest', 'Fashion Underwear and Beach Wear',
       "Men's Clothing", 'Kitchen Furniture and Outdoor Spaces',
       'Industry, Commerce, and Business', 'Fixed Phone',
       'Building To